In [1]:
import sys
sys.path.append('../src/')

In [2]:
%load_ext autoreload
%autoreload 2

from data import load_data
from multilabel.loader import MultiLabelDataset

In [3]:
import os
import copy
import time
import random
import pickle

import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
from tqdm import tqdm

from sklearn.metrics import f1_score, confusion_matrix
from numpy.random import shuffle
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision.io import read_image
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import torch.autograd.profiler as tprofiler
import torch.utils.data as td

plt.rcParams["savefig.bbox"] = 'tight'

In [4]:
seed = 42

random.seed(seed)

# pytorch RNGs
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
np.random.seed(seed)

In [5]:
data_dir = "../data/processed"
images_dir = "../data/processed"

In [6]:
train = pd.read_csv(os.path.join(images_dir,"train.csv"))
val = pd.read_csv(os.path.join(images_dir,"val.csv"))
test = pd.read_csv(os.path.join(images_dir,"test.csv"))

In [7]:
train.head(2)

,Path,Active,Alert,Amazed,Amused,Calm,Cheerful,Confident,Conscious,Creative,...,Educated,Emotional,Fashionable,Feminine,Inspired,Loving,Manly,Persuaded,Thrifty,Youthful
0,6040.jpg,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,159230.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
for idx, row in tqdm(train.iterrows()):
    img = Image.open(os.path.join(data_dir, "images", "train", row.Path))
    img = np.array(img)
    # find number of channels
    if img.ndim != 3:
        print(f"{row.Path} - image has {img.ndim} channel")

11135it [00:29, 380.63it/s]


In [14]:
for idx, row in tqdm(test.iterrows()):
    img = Image.open(os.path.join(data_dir, "images", "test", row.Path))
    img = np.array(img)
    # find number of channels
    if img.ndim != 3:
        print(f"{row.Path} - image has {img.ndim} channel")

3712it [00:10, 367.97it/s]


In [15]:
for idx, row in tqdm(val.iterrows()):
    img = Image.open(os.path.join(data_dir, "images", "val", row.Path))
    img = np.array(img)
    # find number of channels
    if img.ndim != 3:
        print(f"{row.Path} - image has {img.ndim} channel")

3712it [00:09, 386.14it/s]


In [25]:
num_workers = 0
batch_size = 64
image_size = 320

In [26]:
aug = T.Compose([T.Resize(size=(image_size, image_size)),
                 T.ToTensor(),
                 T.Normalize(mean = (0, 0, 0),
                              std  = (1, 1, 1))])


train = pd.read_csv(os.path.join(images_dir, "train.csv")).set_index("Path").fillna(0)
test = pd.read_csv(os.path.join(images_dir, "test.csv")).set_index("Path").fillna(0)
val = pd.read_csv(os.path.join(images_dir, "val.csv")).set_index("Path").fillna(0)


train_dataset = MultiLabelDataset(root=os.path.join(data_dir, "images", "train"),
                                     dataframe=train,
                                     transform=aug)
val_dataset = MultiLabelDataset(root=os.path.join(data_dir, "images", "val"),
                                   dataframe=val,
                                   transform=aug)
test_dataset = MultiLabelDataset(root=os.path.join(data_dir, "images", "test"),
                                    dataframe=test,
                                    transform=aug)

data_loader_train = td.DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=False,
                                  drop_last=False,
                                  num_workers=num_workers,
                                  pin_memory=True)
data_loader_val = td.DataLoader(val_dataset,
                                batch_size=batch_size,
                                shuffle=False,
                                drop_last=False,
                                num_workers=num_workers,
                                pin_memory=True)
data_loader_test = td.DataLoader(test_dataset,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 drop_last=False,
                                 num_workers=num_workers,
                                 pin_memory=True)

In [27]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs, labels in tqdm(data_loader_train):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|██████████| 174/174 [01:05<00:00,  2.65it/s]


In [28]:
count = len(train) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.5989, 0.5510, 0.5175])
std:  tensor([0.3358, 0.3330, 0.3377])


In [29]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs, labels in tqdm(data_loader_val):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|██████████| 58/58 [00:22<00:00,  2.63it/s]


In [30]:
count = len(val) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.5945, 0.5479, 0.5135])
std:  tensor([0.3362, 0.3323, 0.3360])


In [31]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

# loop through images
for inputs, labels in tqdm(data_loader_test):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

100%|██████████| 58/58 [00:21<00:00,  2.68it/s]


In [32]:
count = len(test) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))

mean: tensor([0.6009, 0.5539, 0.5198])
std:  tensor([0.3367, 0.3324, 0.3383])


## Calculating pos_weights

In [36]:
train.sum().values

array([3323., 1489.,  144.,  452.,  233.,  569.,  722.,  437., 5109.,
       6792., 1100.,  137., 5941., 3567., 1342.,  126.,  806.,  450.,
        382.,  363.])

In [38]:
def calculate_pos_weights(class_counts, data):
    pos_weights = np.ones_like(class_counts)
    neg_counts = [len(data)-pos_count for pos_count in class_counts]
    for cdx, (pos_count, neg_count) in enumerate(zip(class_counts,  neg_counts)):
        pos_weights[cdx] = neg_count / (pos_count + 1e-5)

    return torch.as_tensor(pos_weights, dtype=torch.float)

In [39]:
calculate_pos_weights([3323., 1489.,  144.,  452.,  233.,  569.,  722.,  437., 5109.,
       6792., 1100.,  137., 5941., 3567., 1342.,  126.,  806.,  450.,
        382.,  363.], data_loader_train.dataset)

tensor([ 2.3509,  6.4782, 76.3264, 23.6350, 46.7897, 18.5694, 14.4224, 24.4805,
         1.1795,  0.6394,  9.1227, 80.2774,  0.8743,  2.1217,  7.2973, 87.3730,
        12.8151, 23.7444, 28.1492, 29.6749])

In [53]:
import torch
import torch.nn as nn

In [57]:
a = None
b = None
for inp, tar in data_loader_train:
    a = inp
    b = tar
    break

In [ ]:
[2., 3., 6., 6., 3., 7., 9.]

In [64]:
criterion = nn.BCEWithLogitsLoss()
loss = criterion(torch.zeros(64, 7), b)
print(loss)

criterion_weighted = nn.BCEWithLogitsLoss(pos_weight=torch.as_tensor([2., 3., 6., 6., 3., 7., 9.], dtype=torch.float))
loss_weighted = criterion_weighted(torch.zeros(64, 7), b)
print(loss_weighted)

tensor(0.6931)
tensor(0.8912)


In [69]:
criterion = nn.BCEWithLogitsLoss()
loss = criterion(torch.zeros(64, 7), b)
print(loss)

criterion_weighted = nn.BCEWithLogitsLoss(pos_weight=torch.as_tensor(20450*7/25186, dtype=torch.float))
loss_weighted = criterion_weighted(torch.zeros(64, 7), b)
print(loss_weighted)

tensor(0.6931)
tensor(1.1569)
